In [1]:
import numpy as np
base_path='f:/dataset/aspect_attention/data/twitter/'
aspect_id_path=base_path+'aspect_id.txt'
word_id_path=base_path+'word_id.txt'
train_path=base_path+'train.raw'
word2vec_path=base_path+'twitter_word_embedding_partial_100.txt'
test_path=base_path+'test.raw'

In [2]:
def get_batch_index(length,n_iter=100,batch_size=64,is_shuffle=True):
    '''
    每次返回一个batch的index，进行n_iter次
    '''
    index=[i for i in range(length)]
    for i in range(n_iter):
        np.random.shuffle(index)
        for j in range(int(length/batch_size + (1 if length%batch_size else 0))):
                       yield index[j*batch_size:(j+1)*batch_size]


In [3]:
class data():
    def __init__(self):
        self.asp2id=[]
        self.w2v=[]
        self.word2id={}
        self.vocabulary=[]
        self.cnt=0
        self.cnt_2=0
        self.embedding_dim=100
    def load_word2vec(self,wor2vec_path):
        '''
        根据指定的路径取出词向量文件，并遍历词向量，
        生成word2id
        return : dict word2id ,numpy w2v
        '''
        #初始化数据结构w2v
        self.w2v=[[0.]*self.embedding_dim]
        self.word2id={}
        with open(word2vec_path,'r',encoding='utf-8') as file:
            for line in file:
                self.cnt+=1
                sentence=line.lower().split()
                word=sentence[0]
                if len(sentence)!=self.embedding_dim+1:
                    print('a bad word embedding{}'.format(sentence[0]))
                    continue
                self.w2v.append(sentence[1:])
                self.word2id[word]=self.cnt
        #用所有向量的平均值来标识$t$
        self.w2v=np.asarray(self.w2v,dtype=np.float32)
        self.w2v=np.row_stack((self.w2v,np.sum(self.w2v,axis=0)/self.cnt))
        self.cnt+=1
        self.word2id['$t$']=self.cnt
        print('word2vec loadding done !')
        return self.w2v,self.word2id
    def load_word2id_file(self,word2id_path,is_skip=False):
        '''
        将训练集的word2id.txt 读入内存生成训练集的word2id列表
        然后为后面的训练集与词向量进行合并做基础
        生成一个vocabulary文件
        '''
        with open(word2id_path,'r',encoding='utf-8')as file:
            for line in file:
                sentence=line.lower().split()
                self.vocabulary.append(sentence[0])
        return self.vocabulary               
        
    def combine_train_data(self):
        '''
        将词向量生成的词汇表与训练集的词汇表混合在一起
        生成最后的词向量和最后的词汇表
        '''
        for word in self.vocabulary:
            if word not in self.word2id.keys():
                self.cnt+=1
                self.word2id[word]=self.cnt
                self.w2v=np.row_stack((self.w2v,np.random.uniform(-0.01,0.01,(self.embedding_dim,))))
        print('combing done')
        print('word2id 长度为：{}'.format(len(self.word2id)))
        self.w2v=np.asarray(self.w2v,dtype=np.float32)
        return self.w2v,self.word2id
    #到此构造vocabulary,word2id,w2v 三个数据结构的代码完成。
    #下面需要些构造aspect_id的代码
    def load_aspect_id(self,aspect_id_path):
        '''
        从aspect_id_path里面读取aspect目录，
        然后利用求平均值的方法建立aspect词向量
        返回构建好的a2v词向量，以及aspet2id
        a2v: array类型，aspect2id为字典类型
        '''
        self.aspect2id={}
        self.a2v=[]
        self.a2v.append([0.]*self.embedding_dim)
        with open(aspect_id_path,'r',encoding='utf-8') as file:
            for line in file:
                sentence=line.lower().split()
                tmp=[]
                self.cnt_2+=1
                self.aspect2id[' '.join(sentence[:-1])]=self.cnt_2
                for word in sentence:
                    if word in self.word2id.keys():
                       # print(word2id[word])
                        tmp.append(self.w2v[self.word2id[word]])
                if tmp:
                    self.a2v.append(np.sum(tmp,axis=0)/len(tmp))
                else:
                    self.a2v.append(np.random.uniform(-0.01,0.01,(self.embedding_dim,)))
        #print(self.a2v.dtype)
        self.a2v=np.asarray(self.a2v,dtype=np.float32)
        print('aspect_id loading done')
        print('aspect_id 的个数：{}'.format(len(self.a2v)))
        return self.a2v,self.aspect2id
    #构造aspect_id a2v 词向量完成
    #下面构造标签将Y换为onehot编码
    def change_y_2_onehot(self,y):
        '''
        为标签建立onehot编码
        例如-1 [1,0,0]
        0[0,1,0]
        1[0,0,1]
        返回改造后的y
        '''
        self.target=[]
        n_class=set(y)
        n_class=list(n_class)
        n_class.sort()
        print(n_class)
        polarity_map=dict(zip(n_class,range(len(n_class))))
        print(len(y))
        for labels in y:
            tmp=[0]*len(n_class)
            tmp[polarity_map[labels]]=1
            self.target.append(tmp)
        return self.target
                            
        
import random     
test=data()
#test.load_word2vec(word2vec_path)
#test.load_word2id_file(word_id_path)
#test.combine_train_data()
#test.load_aspect_id(aspect_id_path)
#y=[random.randint(-1,1) for i in range(5)]
#print(y)
#test.change_y_2_onehot(y)
test.load_word2vec(word2vec_path)
test.load_word2id_file(word_id_path)
w2v,word2id=test.combine_train_data()
print(w2v.dtype)
a2v,aspect2id=test.load_aspect_id(aspect_id_path)

word2vec loadding done !
combing done
word2id 长度为：12899
float32
aspect_id loading done
aspect_id 的个数：112


In [4]:
class construct_data():
    def __init__(self,file_path):
        self.data_path=file_path
        self.sentence=[]
        self.aspect=[]
        self.y=[]
        self.length=30
    def load_raw_data(self):
        '''
        根据初始化的目录读取sentence，aspect，y，然后返回
        '''
        i=1
        with open(self.data_path,'rb') as file:
            for line in file:
                line=str(line,encoding='utf-8',errors='strict')
                if i==1:
                    tmp=line.strip().lower().split()
                    self.sentence.append(tmp)
                    i+=1
                elif i==2:
                    tmp=line.strip().lower()
                    self.aspect.append(tmp)
                    i+=1
                else:
                    tmp=line.strip().lower().split()
                    self.y.append(int(tmp[0]))
                    i/=3
                #print(line)
    def tokenization_sentence(self,word2id):
        '''
        利用word2id将句子变为id
        返回格式化后的句子
        在对句子进行标记化的时候需要对
        句子进行padding
        '''
        self.input=[]
        for line in self.sentence:
            tmp=[]
            for word in line:
                if word not in word2id.keys():
                    tmp.append(0)
                else:
                    tmp.append(word2id[word])
           # print(tmp)
            if len(tmp)<30:
                length=len(tmp)
                tmp+=[0 for x in range(30-len(tmp))]
            #tmp=tmp[:30]
            #print(len(tmp))
            self.input.append(tmp[:30])
       # print(len(self.input[0]))
        self.input=np.asarray(self.input,dtype=np.int64)
        return self.input
    def tokenization_aspect(self,aspect2id):
        '''
        利用aspect2id 文件将aspect标记化
        返回aspect2id标记化的numpy array
        '''
        self.aspect_id=[]
        for item in self.aspect:
            aspect_name=item.strip().lower()
            if aspect_name not in aspect2id.keys():
                self.aspect_id.append(0)
            else:
                self.aspect_id.append(aspect2id[aspect_name])
        self.aspect_id=np.asarray(self.aspect_id,dtype=np.int64)
        return self.aspect_id
                
        
construct=construct_data(train_path)
construct.load_raw_data()
sentence_token=construct.tokenization_sentence(word2id)
aspect_token=construct.tokenization_aspect(aspect2id)
#print(len(temp_),len(temp_1))
#print(construct.y)
target_token=test.change_y_2_onehot(construct.y)

[-1, 0, 1]
5648


In [5]:
import torch
import torch.nn as nn
print(a2v.dtype)
device=torch.device('cpu')
def construct_tensor(sentence_token,aspect_token,target_token,w2v,a2v,device=device):
        sentence_tensor=torch.tensor(sentence_token,device=device)
        aspect_tensor=torch.tensor(aspect_token,device=device)
        target_tensor=torch.tensor(target_token,device=device)
        w2v=torch.tensor(w2v,device=device)
        a2v=torch.tensor(a2v,device=device)
        return sentence_tensor,aspect_tensor,target_tensor,w2v,a2v
sentence_tensor,aspect_tensor,target_tensor,w2v,a2v=construct_tensor(sentence_token,aspect_token,target_token,w2v,a2v) 
#print(sentence_tensor.dtype)
#数据预处理已经全部完成下一步搭建网络结构

float32


In [48]:
#开始编写网络结构
class lstm_attention(nn.Module):
    def __init__(self,data_num,sentence_len,n_iter,hidden_size,batch_size,embedding_dim,w2v,a2v,p=0):
        super(lstm_attention,self).__init__()
        self.batch_size=batch_size
        self.num_data=data_num
        self.sen_len=sentence_len
        self.hidden_size=hidden_size
        self.embedding_dim=embedding_dim
        self.p=p
        self.attention=torch.empty(self.batch_size,self.embedding_dim)
        self.embedding_sentence=nn.Embedding.from_pretrained(w2v,freeze=False)
        self.embedding_aspect=nn.Embedding.from_pretrained(a2v,freeze=False)
        self.lstm=nn.LSTM(2*self.embedding_dim,self.hidden_size)
        self.l1_hidden=nn.Linear(self.hidden_size,200)
        self.l2_aspect=nn.Linear(self.embedding_dim,200)
        self.softmax=nn.Softmax(1)
        self.linear=nn.Linear((self.hidden_size+self.embedding_dim),3)
        self.logsoft=nn.LogSoftmax(-1)
    def forward(self,sentence_batch,aspect_batch,target_batch):
        '''
        sentence_batch: (batch_size,length(30),1) to (batch_size,length(30),embedding_dim)
        aspect_batch:(batch_size,1) to (batch_size,embedding_dim)
        target_batch:(batch_size,3) to (batch_size,3)
        return result:(batch_size,3)
        '''
        h_0=self.h_0
        c_0=self.c_0
        embedding_sentence=self.embedding_sentence(sentence_batch)#embedding 后（batch_size,length,embedding_dim）
        embedding_aspect=self.embedding_aspect(aspect_batch)#embedding 后 （batch_size,embedding_dim）
        #拼接操作
        #将aspect_id 通过repeat成（batch_size,1,length）
        embedding_aspect=torch.unsqueeze(embedding_aspect,1)
        embedding_aspect_repeat=embedding_aspect.repeat(1,self.sen_len,1)#[batch_size,len,embedding_dim][64,30,100]
        embedding_aspect_repeat.type_as(embedding_sentence)
        #print(embedding_sentence.dtype)
        #print(embedding_aspect_repeat.dtype)
        input_tensor=torch.cat((embedding_sentence,embedding_aspect_repeat),2)#(batch_size,length,2*embedding_dim)
        input_tensor=input_tensor.permute(1,0,2)#[64,30,200]
        #print(input_tensor.shape)
        output,(h_n,c_n)=self.lstm(input_tensor,(h_0,c_0))
        output=output.permute(1,0,2)#[64,30,128][batch_size,length,hidden_size]
        #construct attention weight
        #利用aspect的词向量和每个单词的hidden相结合得到此单词的权重
        #首先对output,aspect进行线性映射
        by_output=self.l1_hidden(output)#[batch_size,len,200]
        by_aspect=self.l2_aspect(embedding_aspect)#[batch_size,1,200]
        by_aspect=by_aspect.permute(0,2,1)#交换维度为tensor相乘做准备[batch_size,200,1]
        att=torch.bmm(by_output,by_aspect)#[batch_size,len,1][64,30,1]
        att=self.softmax(att)
        #aspect与output进行拼接
        result_tensor=torch.cat((output,embedding_aspect_repeat),2)#[batch_size,len,embedding_dim+hidden_size][64,30,228]
        #应用注意力权重
        att=att.permute(0,2,1)#[64,1,30]
        result_tensor=torch.bmm(att,result_tensor)
        result_tensor=self.linear(result_tensor)
        result_tensor=result_tensor.squeeze(1)
        result=self.logsoft(result_tensor)
        return result
    def init_hidden(self):
        self.h_0=torch.empty(1,self.batch_size,self.hidden_size)
        self.c_0=torch.empty(1,self.batch_size,self.hidden_size)
        nn.init.xavier_uniform_(self.h_0)
        nn.init.xavier_uniform_(self.c_0)

In [15]:
import torch.optim as optim
import tqdm 
lr=0.01
data_num=5648
sentence_len=30
n_iter=4
hidden_size=128
batch_size=64
embedding_dim=100
lstm=lstm_attention(data_num,sentence_len,n_iter,hidden_size,batch_size,embedding_dim,w2v,a2v)
lstm.init_hidden()
opt=optim.Adam(lstm.parameters(),lr=1e-3)
loss_func=nn.CrossEntropyLoss()
def cross_entry_one_hot(input_tensor,target_tensor):
    '''
    one_hot 形式 to number 形式转换 
    [1,0,0] 利用 tensor.max()函数
    '''
    _,labels=target_tensor.max(1)
    #print(labels.shape)
    loss=loss_func(input_tensor,labels)
    return loss
for index in tqdm.tqdm(get_batch_index(data_num)):
    #print(sentence_tensor.dtype)
    #print(aspect_tensor.dtype)
    #print(target_tensor[index].shape)
    if target_tensor[index].shape[0]!=64:
        continue
    opt.zero_grad()
    output=lstm(sentence_tensor[index],aspect_tensor[index],target_tensor[index])
    #print(output.shape)
    #print(test)
    loss=cross_entry_one_hot(output,target_tensor[index])
    loss.backward()
    opt.step()
    print('loss:{:.4f}'.format(loss))
    


0it [00:00, ?it/s]

torch.Size([30, 64, 200])
loss:1.0807



1it [00:00,  7.04it/s]

torch.Size([30, 64, 200])
loss:1.0134



2it [00:00,  7.21it/s]

...


8897it [45:07,  2.82it/s]

torch.Size([30, 64, 200])
loss:0.0080



8898it [45:07,  2.83it/s]

torch.Size([30, 64, 200])
loss:0.0303



8899it [45:08,  2.81it/s]
8900it [45:08,  3.29it/s]

In [16]:
#保存模型
torch.save(lstm.state_dict(),'f:/model/lstm_at.pkl')

In [25]:
#验证代码
#取出验证数据集合
construct_val=construct_data(test_path)
#construct=construct_data(train_path)
construct_val.load_raw_data()
sentence_token_val=construct_val.tokenization_sentence(word2id)
aspect_token_val=construct_val.tokenization_aspect(aspect2id)
#print(len(temp_),len(temp_1))
#print(construct.y)
val=data()
target_token_val=val.change_y_2_onehot(construct_val.y)
sentence_val=construct_val.sentence#原始句子信息
print(len(sentence_val))

[-1, 0, 1]
692
692


In [59]:
#构造tensor进行迭代
#利用参数建立新的网络结构进行迭代
lstm_val=lstm_attention(692,30,3,128,1,embedding_dim,w2v,a2v)
#def __init__(self,data_num,sentence_len,n_iter,hidden_size,batch_size,embedding_dim,w2v,a2v,p=0):
lstm_val.init_hidden()
lstm_val.load_state_dict(torch.load('f:/model/lstm_at.pkl'))
sentence_tensor_val,aspect_tensor_val,target_tensor_val,_,_=construct_tensor(sentence_token_val,aspect_token_val,target_token_val,w2v,a2v)
correct_num=0
error_num=0
tag=['negative','netural','positive']
for i in range(692):
    sent=sentence_val[i]
    input_tensor=sentence_tensor[i].unsqueeze(0)
    input_aspect_tensor=aspect_tensor_val[i].unsqueeze(0)
    print(input_tensor.shape)
    with torch.no_grad():
        output=lstm_val(input_tensor,input_aspect_tensor,target_tensor_val)
        #output=output.view(1,3)
        print(output.shape)
        _,labels=torch.max(output,1)
        print(labels)
       # print(target_tensor_val[i])
        _,labels_acu=torch.max(target_tensor_val[i],0)
        #print(labels_acu)
        num_l=int(labels.item())
        num_l_ac=int(labels_acu.item())
        print('sentence:{}'.format(sent))
        print('predict:{}   actual:{}'.format(tag[num_l],tag[num_l_ac]))
        if num_l==num_l_ac:
            correct_num+=1
        else:
            error_num+=1
print('accuracy:{:.4f}'.format(correct_num/692))
    

E:\xcl\program\anaconda\lib\site-packages\ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
E:\xcl\program\anaconda\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['$t$', 'to', 'miss', '3rd', 'straight', 'playoff', 'game', '|', 'the', '...', ':', '$t$', 'will', 'miss', 'his', 'third', 'straight', 'play', '...', '.']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['dear', '$t$', ',', 'gray', 'hoodies', 'turned', 'into', 'leather', 'jackets', ',', "''", 'ay', '!', "''", 'turned', 'into', "''", 'swag', "''", ',', 'u', 'grew', 'up', ',', 'and', 'we', "'ve", 'been', 'here', 'all', 'the', 'way', '.', 'rt', 'i', 'u', 'love', 'bieber', '♥']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['received', 'my', '$t$', 'account', 'today', '!', 'sorry', 'have', 'no', 'invites', ',', 'but', 'i', 'will', 'spread', 'the', 'love', 'if', 'i', 'receive', 'any', ',', 'thanks', 'twitter', 'community', '!']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size

tensor(2)
sentence:['✔', 'acting', 'skills', '✔', 'great', 'voice', '✔', 'nice', 'hair', '✔', 'hilarious', '✔', 'adorable', '✔', 'talented', '✔', 'gorgeous', '=', 'perfect', 'and', 'that', 'must', 'be', '$t$', '!', '<', '3']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['waiting', 'for', '$t$', 'james', 'franco', 'is', 'but', 'that', 'video', 'of', 'the', 'chick', 'on', 'jay', 'keno', 'was', 'funny', 'the', 'gardners', 'face', 'was', 'priceless']
predict:negative   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['miley', 'cyrus', 'is', 'another', 'petulant', 'girl', 'like', '$t$', ',', 'i', 'love', 'dakota', 'fanning', 'is', 'another', 'type', 'of', 'person', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['twitter', 'question', ':', 'whose', 'made', 'more', 'bad', 'movies', '...', '$t$', 'or', 'jennifer', 'anis

sentence:['interesting', 'alternative', '$t$', 'interface', '-lsb-', 'thenextweb.com', '-rsb-', '-lsb-', 'shared', 'by', '12', 'people', '-rsb-', '-lrb-', 'so', '.', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['$t$', "'", 'music', 'has', 'gotten', 'progressively', 'worse', '.', 'she', 'sings', 'less', 'and', 'less', 'and', 'relies', 'upon', 'melodyne', 'to', 'make', 'her', 'boice', 'sound', 'like', 'crap', '!']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['i', 'want', 'a', '$t$', 'damn', 'it', '!']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['is', 'obama', 'trying', 'to', 'be', 'just', 'like', '$t$', '??', 'he', "'s", 'doing', 'a', 'pretty', 'good', 'job', 'so', 'far', '...', 'icon', '|', 'he', 'has', 'passed', 'jimmy', '!!']
predict:netural   actual:netural
torch.Size([1, 30])
torch

sentence:['help', 'me', 'a', 'problem', 'with', '$t$', 'ultimate', '?', ':', 'i', 'have', 'an', 'alfa', 'wireless', 'adepdter', 'and', 'its', 'usb', 'awush32h', '-lrb-', 'something', 'look', '.', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['songs', 'of', '$t$', 'nowadays', 'are', 'stupid', '...', 'non', 'sense', 'things', '...', 'it', "'s", 'all', 'about', 'sex', '!!', 'geeezzz', '...']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['$t$', 'was', 'awesome', '!', ':', 'd']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['one', 'thing', 'i', 'hate', 'about', 'being', 'a', 'sports', 'fan', 'is', 'that', 'it', 'hurts', 'every', 'time', 'you', 'realize', 'your', 'team', 'is', 'done', '...', 'being', 'a', '$t$', 'fan', 'is', 'a', 'buffer', 'lol']
predict:netural   actual:netural
torch.Size([1, 3

...

torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['so', 'you', 'dissed', '$t$', 'but', 'you', 'are', 'watching', 'ashlee', 'simpson', '.', 'you', 'fail', 'as', 'a', 'human', 'being', '.']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['ha', 'i', 'love', 'that', 'movie', 'where', '$t$', 'does', 'the', 'twins', '...', 'what', 'was', 'the', 'name', 'of', 'that', 'movie', '?!?']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['dear', '$t$', ':', 'love', 'that', 'your', '2-cd', 'set', 'includes', '1', 'that', "'s", 'safe', 'for', 'kids', '&', '1', 'that', "'s", 'clearly', 'not', '.', 'planned', 'move', 'or', 'accidental', 'genius', '?']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['i', 'think', 'its', 'pretty', 'cool', '.', 'went', 'with', 'a', 'friend', 'of', 'mine', 'who'

sentence:['joining', 'the', 'vgp', 'team', 'again', 'tonight', '.', 'got', 'a', 'lot', 'to', 'talk', 'about', 'including', 'red', 'steel', '2', ',', 'heavy', 'rain', ',', 'new', 'super', 'mario', 'brothers', '$t$', 'and', 'more', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['i', 'really', 'wished', 'peru', 'had', 'knocked', 'off', 'argentina', 'today', '.', '.', 'i', 'would', 'love', 'to', 'see', '$t$', 'get', 'fired', '.']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['sitting', 'in', 'the', 'library', 'studying', 'some', 'gi', 'tract', 'and', 'fun', 'stuff', '...', 'it', 'literally', 'is', 'shit', 'ha', '.', 'cant', 'wait', 'to', 'play', 'some', '$t$', 'in', 'like', 'an', 'hour', 'though', '...']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['dear', '$t$', ',', 'i', 'would', 'like', 't

sentence:['hey', ',', 'spambot', '!', 'i', 'know', 'that', "'s", 'not', '$t$', 'in', 'that', 'photo', '.', 'britney', 'is', "n't", 'anywhere', 'near', 'that', 'good', 'looking', '.']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['playing', 'on', 'the', '$t$', 'fit', 'with', 'my', 'mum', ',', 'its', 'hilarious', ':', 'p']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['lmfao', '!', 'its', 'more', 'like', '$t$', '!', 'do', "n't", 'worry', 'since', 'hween', 'falls', 'on', 'a', 'sat', 'i', 'can', 'actually', 'celebrate', 'it', 'with', 'you', '!!', 'let', "'s", 'party', '?']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['lekker', 'relaxed', '$t$', 'and', 'the', 'half', 'blood', 'prince', 'aan', "'", 't', 'kijken', '!']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1

torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['nowplaying', "''", 'ai', "n't", 'no', 'fun', '-lrb-', 'if', 'the', 'homies', 'ca', "n't", 'have', 'none', '-rrb-', "''", '-', '$t$', "'", 'cause', 'iiiiii', 'never', 'met', 'a', 'girl', 'that', 'i', 'love', 'in', 'this', 'whole', 'wide', 'world', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:["''", '$t$', 'has', 'opened', 'the', 'first', 'new', 'military', 'action', 'of', 'his', 'presidency', 'in', 'libya', '.', "''", '.', '/', 'wrong', '.', 'he', 'killed', '41', 'civilians', 'in', 'yemen']
predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['i', 'hate', 'when', 'people', 'think', 'its', 'weird', 'that', 'i', 'listen', 'to', '$t$', ',', 'paramore', ',', 'or', 'anything', 'pop', '&', 'rock', '...', 'am', 'i', 'not', 'a', 'pop', '&', 'rock', 'type', 'a', 'chick', '?']
predict:positive 

predict:negative   actual:negative
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 0, 1])
tensor(2)
sentence:['i', 'voted', 'moldova', 'for', "''", '$t$', 'birthday', 'kiss', "''", 'vote', '!', '.']
predict:positive   actual:positive
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['rttrudyvan', 'wyk', '$t$', '360', 'microsoft', 'superior', 'aftersales', 'service', ',']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([0, 1, 0])
tensor(1)
sentence:['$t$', 'gets', 'true', 'software', 'base', 'station', 'before', 'mac', 'os', 'x', ':', 'i', 'dislike', 'turning', 'comparisons', 'between', 'windows', 'and', 'ma', '.', '.']
predict:netural   actual:netural
torch.Size([1, 30])
torch.Size([1, 3])
tensor([1, 0, 0])
tensor(0)
sentence:['flex', 'im', 'disappointed', 'in', 'photos', 'you', 'have', 'laid', 'out', 'not', 'enough', 'sexy', 'and', 'you', 'put', 'up', '$t$', 'and', 'teyana', 'taylor', 'as', 'sexy', '?']
predict:negative   act